## Common preparations

### Importing necessary libs.

In [1]:
import pandas as pd
import os

### Defining paths to data files.

In [2]:
base_dir = os.path.join('data', 'bookmaker')
log_filename = os.path.join(base_dir, 'log.csv')
users_filename = os.path.join(base_dir, 'users.csv')

### Loading log and users data from CSV files into corresponding Pandas DataFrames and merging them.

In [26]:
#Функция, редактирующую значения столбца win
def fillna_win(row):
    if row.bet==0: #если ставка bet=0, то выигрыш win=0
        return 0
    elif row.bet>0 and row.win>0: #если ставка bet>0 и выигрыш win>0, то выигрыш win=win
        return row.win
    else:
        return -row.bet #если ставка bet>0 и выигрыш win=0, то выигрыш win=-bet
#Функция, формирующая значения столбца net
def column_net(row):
    if row.win<0: #если выигрыш win<0, то net=win
        return row.win
    else: #если выигрыш win>=0, то net=win-bet
        return row.win-row.bet
    
#Готовим данные
#log
log = pd.read_csv(log_filename, header=None)
log.columns = ['user_id','time','bet','win']

log.bet = log.bet.fillna(0)
log.win = log.apply(lambda row: fillna_win(row), axis=1)
log = log[log.user_id != '#error']
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])
log['net'] = log.apply(lambda row: column_net(row), axis=1)
log['time'] = log['time'].apply(lambda x: x[1:])
log.time = pd.to_datetime(log['time'])

#users
us = pd.read_csv(users_filename,sep='\t', encoding='koi8_r')
us.columns = ['user_id','mail','city']
us.user_id = us.user_id.apply(lambda x: x.lower())

#Слияние
log_users = pd.merge(log, us, on='user_id')

## Tasks.

### Task 1.
Используйте датасет, который получился в результате всех преобразований выше (в том числе, заполнение пропусков). Ответ запишите в поле ниже в виде целого числа (отбросьте дробную часть).
Посчитайте медиану баланса по каждому пользователю. Для этого сгруппируйте по пользователям, возьмите признак net, просуммируйте по каждому пользователю и получите медиану.

### Solution:

In [27]:
log_users.groupby('user_id')['net'].sum().median()

1986.0

### Conclusion: 1986 is the right answer!!!

### Task 2.
Используйте датасет, который получился в результате всех преобразований выше (в том числе, заполнение пропусков). Ответ запишите в поле ниже в виде целого числа (отбросьте дробную часть).
Сколько раз в среднем каждый человек приходит, не делая ставок, при условии, что у этого человека все-таки есть хотя бы одна ставка? Например: Человек посетил букмекерскую контору 5 раз из них 1 раз сделал ставку, 4 раза нет - условие выполняется. Человек посетил букмекерскую контору 5 раз из них ни разу ставку не сделал - условие не выполняется. Для того, чтобы узнать это, просуммируйте в каждой группе количество записей со ставкой, равной 0, и поделите на общее количество групп. Если при этом в группе нет записей со ставкой больше 0, считаем количество записей в данной группе равным 0.

### Solution:

In [28]:
log_users[log_users['bet'] == 0].groupby('user_id')['bet'].count().mean()

5.05

### Another solution:

In [29]:
gr = log_users.groupby('user_id').filter(lambda u:u.bet.max()>0).groupby('user_id')
counter = lambda u: u[u.bet==0].bet.count()
result = gr.apply(counter).mean()
print(result)

5.05


### Conclusion: 5 will be the right answer!!!

### Task 3.
Используйте датасет, который получился в результате всех преобразований выше (в том числе, заполнение пропусков). Ответ запишите в поле ниже в виде целого числа - количества дней.
Сколько в среднем времени проходит между появлением человека в сервисе и первой ставкой? Считать нужно только тех, кто делал ставку. Для того, чтобы узнать это, напишите метод, считающий минимальное время среди ставок, равных 0, и минимальное время среди ставок больше 0. После этого верните разницу между вторым и первым числом. Пройдитесь по всем группам в цикле. Если в группе нет ставок больше 0, пропустите эту группу. Просуммируйте разницу во времени для каждой группы (с помощью метода, описанного выше) и поделите на количество групп, которые вы не пропустили.

In [71]:
def first_bet_time_gape(df, user):
    user_entries = df[df['user_id'] == user]
    user_zero_bets = user_entries[user_entries['bet'] == 0]
    first_visit_time = user_zero_bets['time'].sort_values().iloc[0]
    user_real_bets = user_entries[user_entries['bet'] != 0]
    first_bet_time = user_real_bets['time'].sort_values().iloc[0]
    return first_bet_time - first_visit_time

In [77]:
users = log_users['user_id'].unique()
first_bet_times = []
for user in users:
    time_delta = first_bet_time_gape(log_users, user)
#     if time_delta.days > 0:
    first_bet_times.append(time_delta.days)

In [78]:
sum(first_bet_times)/len(first_bet_times)

45.79